In [ ]:
#Importando a biblioteca
import pandas as pd

## Atletas (Pré-processamento)

In [ ]:
#Comando para carregar o dataset. Se o seu dataset for em .csv, utilize pd.read_csv(...)
atletas_original = pd.read_excel('Athletes.xlsx')

#Comando 'head' mostra as 5 primeiras linhas da tabela
atletas_original.head()

,Name,NOC,Discipline
0,AALERUD Katrine,Norway,Cycling Road
1,ABAD Nestor,Spain,Artistic Gymnastics
2,ABAGNALE Giovanni,Italy,Rowing
3,ABALDE Alberto,Spain,Basketball
4,ABALDE Tamara,Spain,Basketball


In [ ]:
#Comando 'describe' mostra algumas informações sobre o dataset
atletas_original.describe()

,Name,NOC,Discipline
count,11085,11085,11085
unique,11062,206,46
top,HALL James,United States of America,Athletics
freq,2,615,2068


In [ ]:
#Esse comando pega todas as informações faltantes e retorna a soma delas. Como pode ver, pra esse dataset não há dados faltantes
atletas_original.isna().sum()

Name          0
NOC           0
Discipline    0
dtype: int64

In [ ]:
#Aqui foi só para criar a coluna de 'Id' com base no index da própria tabela
atletas_original["Id"] = atletas_original.index

In [ ]:
atletas_original.head()

,Name,NOC,Discipline,Id
0,AALERUD Katrine,Norway,Cycling Road,0
1,ABAD Nestor,Spain,Artistic Gymnastics,1
2,ABAGNALE Giovanni,Italy,Rowing,2
3,ABALDE Alberto,Spain,Basketball,3
4,ABALDE Tamara,Spain,Basketball,4


In [ ]:
#Comando 'drop' remove as colunas que vc passa
atletas = atletas_original.drop(columns=["NOC","Discipline"])

#Comando 'rename' renomeia uma coluna
atletas = atletas.rename({"Name":"Nome"}, axis=1)

#Comando 'reindex' muda a ordem das colunas
atletas = atletas.reindex(columns=["Id","Nome"])

In [ ]:
atletas["Ano"] = None;

In [ ]:
atletas.head()

,Id,Nome,Ano
0,0,AALERUD Katrine,None
1,1,ABAD Nestor,None
2,2,ABAGNALE Giovanni,None
3,3,ABALDE Alberto,None
4,4,ABALDE Tamara,None


## Requests

In [ ]:
from lxml import html
import requests

In [ ]:
# Função que pega o ano de nascimento de cada atleta a partir do site oficial das olimpíadas
def findYear(row):
  try:
    nomeSplit = row.Nome.split()
    index = 0
    for i in range(len(nomeSplit)):
      if nomeSplit[i].upper() != nomeSplit[i]:
        index = i
        break;
    nomeOrd = nomeSplit[index:]+nomeSplit[:index]
    for i in range(len(nomeOrd)):
      nomeOrd[i] = nomeOrd[i].lower()

    nome = "-".join(nomeOrd)

    r = requests.get("https://olympics.com/en/athletes/"+nome)
    tree = html.fromstring(r.content)
    details = tree.xpath('//ul[@class="detail__list"]/li/div/text()')
    b = details.index("Year of Birth")
    year = details[b+1]
    row.Ano = year
  except ValueError:
    print(row.Nome)
  return row

atletas = atletas.apply(lambda x: findYear(x),axis=1)

ABALDE Alberto
ABDALLA Maryam
ABDALLAH Shahd
ABDELAZIZ Farah
ABDELMAWGOUD Mohamed
ABLIAZIN Denis
ABRAMYAN Benik
ABRINES REDONDO Alejandro
ACHANTA Kamal
ACOSTA Marcelo
ADAMUS Bartlomiej Stejan
ADAO de SA Mateus Daniel
ADEBAYO Edrice Femi
AFRIAT Charlotte
AGUIAR Mayra
AGUILAR Evelis Jazmin
AGUNDEZ GARCIA Gabriela
AICOBOAE Vlad
AJAYI Yinka
AKHMETOVA Tatyana
AL KHATRI Hamed
AL-KAABI Fatimah Abbas Waheeb
AL-WIR Amro
AL-YAMANI Mokhtar
ALBA CASTILLO Rafael Yunier
ALEGRE BIOSCA David
ALEXANDRI Eirini
ALKASBEH Obada Mohammad Mustafa
ALMAYA Haimro
ALMEIDA Teresa
ALMUTAIRI Saeid
ALVAREZ Lazaro
ALY ELKAMASH Marwan
AMADOR Andrey
AMARIS Cesar
AMBAKA Willy
AMIT
AMORIM TALESKA Eduarda
AMUSAN Tobi
ANAGNOSTOPOULOU Chrysoula
ANASTASAKIS Michail
ANCER Abraham
ANDRESSINHA
ANDREW Michael
ANDREWS-NAHU Kanah
APPEL CHEUICHE Marcio
ARAKAKI Maria Eduarda
ARANGO ESTRADA Monica
ARCEO Mariana
ARCILA HURTADO Isabella
ARENAS Sandra Lorena
AREVALO Carlos
ARMSTRONG Joseph
ARREGUI Brian Agustin
ARROYO Jarod
ARTACHO del 

In [ ]:
atletas

,Id,Nome,Ano
0,0,AALERUD Katrine,1994
1,1,ABAD Nestor,1993
2,2,ABAGNALE Giovanni,1995
3,3,ABALDE Alberto,None
4,4,ABALDE Tamara,1989
...,...,...,...
11080,11080,ZWICKER Martin Detlef,None
11081,11081,ZWOLINSKA Klaudia,None
11082,11082,ZYKOVA Yulia,1995
11083,11083,ZYUZINA Ekaterina,1996


In [ ]:
atletas.isna().sum()

Id         0
Nome       0
Ano     1225
dtype: int64

In [ ]:
atletas.to_csv("atletas.csv",index=False)

In [ ]:
atletasEsportes = pd.DataFrame(columns=["Nome","NomeUrl","Esporte","Modalidade","Medalha"])

In [ ]:
# Trecho de código que realiza requisições ao site "olympics.com" e obtém
# ano de nascimento do atleta e modalidades em que participou
for i in range(len(atletas)):
  atleta = atletas.loc[i]
  nome=atleta.Nome.replace(".","").replace("'"," ")
  nomeSplit = nome.split()

  index = 0
  for j in range(len(nomeSplit)-1,-1,-1):
    if nomeSplit[j].upper() == nomeSplit[j] and j!=len(nomeSplit)-1:
      index = j+1
      break;
  nomeOrd = nomeSplit[index:]+nomeSplit[:index]
  for j in range(len(nomeOrd)):
    nomeOrd[j] = nomeOrd[j].lower()

  nomeUrl = "-".join(nomeOrd)
  nomeReal = " ".join(nomeOrd).title()
  
  try:
    r = requests.get("https://olympics.com/en/athletes/"+nomeUrl)
    tree = html.fromstring(r.content)

    details = tree.xpath('//ul[@class="detail__list"]/li/div/text()')
    b = details.index("Year of Birth")
    year = details[b+1]
    atletas.loc[i,"Ano"] = year

    tabela_part = tree.xpath('//table[@class="sm-mb6 has-header"]/tbody/tr')
    
    nomeOlimp = tabela_part[0].xpath('.//h2/text()')
    if len(nomeOlimp)>0:
      if nomeOlimp[0] == "Tokyo 2020":
        for tr in tabela_part:
          td = tr.xpath('.//td')
          medalha = td[1].xpath('.//div')[0].text_content().replace('"','').replace("\n","").replace("\r","").replace(" ","")
          modalidade = td[2].text_content()
          esporte = td[3].text_content()
          atletasEsportes.loc[len(atletasEsportes)] = [nomeReal, nomeUrl, esporte, modalidade, medalha]
  except:
    print(nomeReal)

Alberto Abalde
Maryam Abdalla
Shahd Abdallah
Farah Abdelaziz
Mohamed Abdelmawgoud
Uta Abe
Denis Abliazin
Benik Abramyan
Alejandro Abrines Redondo
Kamal Achanta
Marcelo Acosta
Bartlomiej Stejan Adamus
Mateus Daniel Adao De Sa
Edrice Femi Adebayo
Charlotte Afriat
Mayra Aguiar
Evelis Jazmin Aguilar
Gabriela Agundez Garcia
Mohamed Ahmed
Vlad Aicoboae
Yinka Ajayi
Tatyana Akhmetova
Hamed Al Khatri
Fatimah Abbas Waheeb Al-Kaabi
Amro Al-Wir
Mokhtar Al-Yamani
Rafael Yunier Alba Castillo
David Alegre Biosca
Eirini Alexandri
Mohamed Ali
Mohamed Ali
Obada Mohammad Mustafa Alkasbeh
Haimro Almaya
Teresa Almeida
Saeid Almutairi
Lazaro Alvarez
Dani Alves
Marwan Aly Elkamash
Andrey Amador
Cesar Amaris
Willy Ambaka
Amit
Eduarda Amorim Taleska
Tobi Amusan
Chrysoula Anagnostopoulou
Michail Anastasakis
Abraham Ancer
Andressinha
Michael Andrew
Kanah Andrews-Nahu
Marcio Appel Cheuiche
Maria Eduarda Arakaki
Monica Arango Estrada
Mariana Arceo
Isabella Arcila Hurtado
Sandra Lorena Arenas
Carlos Arevalo
Joseph 

In [ ]:
# Trecho de código que realiza requisições ao site "olympics.com" e obtém
# ano de nascimento do atleta e modalidades em que participou
for i in range(len(atletas)):
  ###
   # Tratamento do nome do atleta
  ###
  
  try:
    r = requests.get("https://olympics.com/en/athletes/"+nomeUrl)
    tree = html.fromstring(r.content)

    details = tree.xpath('//ul[@class="detail__list"]/li/div/text()')
    b = details.index("Year of Birth")
    year = details[b+1]
    atletas.loc[i,"Ano"] = year

    tabela_part = tree.xpath('//table[@class="sm-mb6 has-header"]/tbody/tr')
    
    nomeOlimp = tabela_part[0].xpath('.//h2/text()')
    if len(nomeOlimp)>0:
      if nomeOlimp[0] == "Tokyo 2020":
        for tr in tabela_part:
          td = tr.xpath('.//td')
          medalha = td[1].xpath('.//div')[0].text_content().replace('"','').replace("\n","").replace("\r","").replace(" ","")
          modalidade = td[2].text_content()
          esporte = td[3].text_content()
          atletasEsportes.loc[len(atletasEsportes)] = [nomeReal, nomeUrl, esporte, modalidade, medalha]
  except:
    print(nomeReal)

In [ ]:
exemplo = atletasEsportes.drop(columns=["Unnamed: 0"])
exemplo.loc[100:120]

,Nome,NomeUrl,Esporte,Modalidade,Medalha
100,Valentina Acosta Giraldo,valentina-acosta-giraldo,Archery,Mixed Team,#26
101,Valentina Acosta Giraldo,valentina-acosta-giraldo,Archery,Women's Individual,#=33
102,Yenny Acuna,yenny-acuna,Football,Women,#11
103,Kazuya Adachi,kazuya-adachi,Canoe Slalom,Men's Kayak,#16
104,Seiya Adachi,seiya-adachi,Water Polo,Men,#10
105,Amal Adam,amal-adam,Archery,Mixed Team,#29
106,Amal Adam,amal-adam,Archery,Women's Individual,#=33
107,Constantin Adam,constantin-adam,Rowing,Men's Eight,#7
108,Klaudia Adamek,klaudia-adamek,Athletics,Women's 4 x 100m Relay,#n/a
109,Patrycja Adamkiewicz,patrycja-adamkiewicz,Taekwondo,Women -57kg,#=11


In [ ]:
atletas.isna().sum()

Id         0
Nome       0
Ano     1009
dtype: int64

In [ ]:
atletas.to_csv("atletas_idades.csv")

In [ ]:
atletasEsportes.to_csv("atletasEsportes.csv")

## Tabela 1 - Edição dos Jogos

In [ ]:
edicaoDosJogos = pd.DataFrame(columns=["Ano","NumerodaEdicao","Cidade","TotaldeAtletas","Mascote"])
edicaoDosJogos.loc[0] = ["2020","30","Tokyo","11085","Miraitowa"]
edicaoDosJogos

,Ano,NumeroDaEdicao,CidadeSede,TotalDeAtletas,Mascote
0,2020,30,Tokyo,11085,Miraitowa


In [ ]:
edicaoDosJogos.to_csv("edicoes-Tokyo.csv")

## Tabela 2 - Atleta

In [ ]:
atletas = pd.read_csv("atletas_idades.csv",dtype=str)
atletas = atletas.drop(columns=["Unnamed: 0"])
atletas = atletas.rename({"Ano":"AnoDeNascimento"}, axis=1)
atletas

,Id,Nome,AnoDeNascimento
0,0,AALERUD Katrine,1994
1,1,ABAD Nestor,1993
2,2,ABAGNALE Giovanni,1995
3,3,ABALDE Alberto,NaN
4,4,ABALDE Tamara,1989
...,...,...,...
11080,11080,ZWICKER Martin Detlef,NaN
11081,11081,ZWOLINSKA Klaudia,NaN
11082,11082,ZYKOVA Yulia,1995
11083,11083,ZYUZINA Ekaterina,1996


In [ ]:
for i in range(len(atletas)):
  atleta = atletas.loc[i]
  nome=atleta.Nome.replace(".","").replace("'"," ")
  nomeSplit = nome.split()

  index = 0
  for j in range(len(nomeSplit)-1,-1,-1):
    if nomeSplit[j].upper() == nomeSplit[j] and j!=len(nomeSplit)-1:
      index = j+1
      break;
  nomeOrd = nomeSplit[index:]+nomeSplit[:index]
  for j in range(len(nomeOrd)):
    nomeOrd[j] = nomeOrd[j].lower()

  nomeReal = " ".join(nomeOrd).title()
  atletas.loc[i,"Nome"] = nomeReal

atletas.head()

,Id,Nome,AnoDeNascimento
0,0,Katrine Aalerud,1994
1,1,Nestor Abad,1993
2,2,Giovanni Abagnale,1995
3,3,Alberto Abalde,NaN
4,4,Tamara Abalde,1989


In [ ]:
atletas["Sexo"] = None

In [ ]:
atletasEsportes = pd.read_csv("atletasEsportes.csv")
atletasEsportes.head(60)

,Unnamed: 0,Nome,NomeUrl,Esporte,Modalidade,Medalha
0,0,Katrine Aalerud,katrine-aalerud,Cycling Road,Women's Individual Time Trial,#20
1,1,Katrine Aalerud,katrine-aalerud,Cycling Road,Women's Road Race,#37
2,2,Nestor Abad,nestor-abad,Artistic Gymnastics,Men's All-Around,#n/a
3,3,Nestor Abad,nestor-abad,Artistic Gymnastics,Men's Floor Exercise,#n/a
4,4,Nestor Abad,nestor-abad,Artistic Gymnastics,Men's Horizontal Bar,#n/a
5,5,Nestor Abad,nestor-abad,Artistic Gymnastics,Men's Parallel Bars,#n/a
6,6,Nestor Abad,nestor-abad,Artistic Gymnastics,Men's Pommel Horse,#n/a
7,7,Nestor Abad,nestor-abad,Artistic Gymnastics,Men's Rings,#n/a
8,8,Nestor Abad,nestor-abad,Artistic Gymnastics,Men's Team,#n/a
9,9,Nestor Abad,nestor-abad,Artistic Gymnastics,Men's Vault,#n/a


In [ ]:
atletasEsportes.head(60)

,Unnamed: 0,Nome,NomeUrl,Esporte,Modalidade,Medalha
0,0,Katrine Aalerud,katrine-aalerud,Cycling Road,Women's Individual Time Trial,#20
1,1,Katrine Aalerud,katrine-aalerud,Cycling Road,Women's Road Race,#37
2,2,Nestor Abad,nestor-abad,Artistic Gymnastics,Men's All-Around,#n/a
3,3,Nestor Abad,nestor-abad,Artistic Gymnastics,Men's Floor Exercise,#n/a
4,4,Nestor Abad,nestor-abad,Artistic Gymnastics,Men's Horizontal Bar,#n/a
5,5,Nestor Abad,nestor-abad,Artistic Gymnastics,Men's Parallel Bars,#n/a
6,6,Nestor Abad,nestor-abad,Artistic Gymnastics,Men's Pommel Horse,#n/a
7,7,Nestor Abad,nestor-abad,Artistic Gymnastics,Men's Rings,#n/a
8,8,Nestor Abad,nestor-abad,Artistic Gymnastics,Men's Team,#n/a
9,9,Nestor Abad,nestor-abad,Artistic Gymnastics,Men's Vault,#n/a


In [ ]:
for i in range(len(atletas)):
  atleta = atletas.loc[i]
  esportesAtleta = atletasEsportes[atletasEsportes.Nome==atleta.Nome]
  esportesAtleta = esportesAtleta.reset_index(drop="True")
  for j in range(len(esportesAtleta)):
    if "Women" in esportesAtleta.loc[j,"Modalidade"]:
      atletas.loc[i,"Sexo"]="F"
      break;
    elif "Men" in esportesAtleta.loc[j,"Modalidade"]:
      atletas.loc[i,"Sexo"]="M"
      break;

In [ ]:
atletas.isna().sum()

Id                    0
Nome                  0
AnoDeNascimento    1009
Sexo               1950
dtype: int64

In [ ]:
atletas.head(15)

,Id,Nome,AnoDeNascimento,Sexo
0,0,Katrine Aalerud,1994,F
1,1,Nestor Abad,1993,M
2,2,Giovanni Abagnale,1995,M
3,3,Alberto Abalde,NaN,None
4,4,Tamara Abalde,1989,F
5,5,Luc Abalo,1984,M
6,6,Cesar Abaroa,1996,M
7,7,Abobakr Abass,1998,M
8,8,Hamideh Abbasali,1990,F
9,9,Islam Abbasov,1996,M


In [ ]:
atletas = atletas.fillna("-")
atletas.to_csv("atletas-Tokyo.csv")

## Tabela 3 - Comitê Olímpico

In [ ]:
paises = pd.read_csv("comites.csv")
paises = paises.drop(columns=["Unnamed: 0"])
paises = paises.rename({"Unnamed: 1":"Sigla"}, axis=1)
paises = paises.rename({"Unnamed: 2":"País"}, axis=1)
paises.loc[len(paises)] = ["ROC","Russian Olympic Committee"]
paises.loc[len(paises)] = ["EOR","Refugee Olympic Team"]
paises = paises.sort_values("Sigla")
paises = paises.reset_index(drop=True)
paises

,Sigla,País
0,AFG,Afghanistan
1,AHO,Netherlands Antilles
2,ALB,Albania
3,ALG,Algeria
4,AND,Andorra
...,...,...
208,VIE,Vietnam
209,VIN,Saint Vincent and the Grenadines
210,YEM,Yemen
211,ZAM,Zambia


In [ ]:
for i in range(len(atletas_original)):
  atleta = atletas_original.loc[i]
  pais = paises[paises.País==atleta.NOC]
  pais = pais.reset_index(drop="True")
  if len(pais)>0:
    atletas_original.loc[i,"Comite"]=pais.loc[0].Sigla

In [ ]:
atletas_original.isna().sum()

Name             0
NOC              0
Discipline       0
Id               0
Comite        1773
dtype: int64

In [ ]:
atletas_original[atletas_original.Comite.isna()]

,Name,NOC,Discipline,Id,Comite
8,ABBASALI Hamideh,Islamic Republic of Iran,Karate,8,NaN
12,ABBOTT Monica,United States of America,Baseball/Softball,12,NaN
29,ABDIRAHMAN Abdi,United States of America,Athletics,29,NaN
47,ABEDINI Mojtaba,Islamic Republic of Iran,Fencing,47,NaN
56,ABLIAZIN Denis,ROC,Artistic Gymnastics,56,NaN
...,...,...,...,...,...
11068,ZOLOTIC Anastasija,United States of America,Taekwondo,11068,NaN
11071,ZOU Jingyuan,People's Republic of China,Artistic Gymnastics,11071,NaN
11074,ZUEV Alexander,ROC,3x3 Basketball,11074,NaN
11082,ZYKOVA Yulia,ROC,Shooting,11082,NaN


In [ ]:
for i in range(len(atletas_original)):
  atleta = atletas_original.loc[i]
  if (atleta.NOC=="United States of America"):
    atletas_original.loc[i,"Comite"]="USA"
  elif (atleta.NOC=="People's Republic of China"):
    atletas_original.loc[i,"Comite"]="CHN"
  elif (atleta.NOC=="ROC"):
    atletas_original.loc[i,"Comite"]="ROC"
  elif (atleta.NOC=="Islamic Republic of Iran"):
    atletas_original.loc[i,"Comite"]="IRI"
  elif (atleta.NOC=="Republic of Korea"):
    atletas_original.loc[i,"Comite"]="KOR"
  elif (atleta.NOC=="Congo"):
    atletas_original.loc[i,"Comite"]="COD"
  elif (atleta.NOC=="Chinese Taipei"):
    atletas_original.loc[i,"Comite"]="TPE"
  elif (atleta.NOC=="Syrian Arab Republic"):
    atletas_original.loc[i,"Comite"]="SYR"
  elif (atleta.NOC=="Democratic Republic of Timor-Leste"):
    atletas_original.loc[i,"Comite"]="TLS"
  elif (atleta.NOC=="Republic of Moldova"):
    atletas_original.loc[i,"Comite"]="MDA"
  elif (atleta.NOC=="Côte d'Ivoire"):
    atletas_original.loc[i,"Comite"]="CIV"
  elif (atleta.NOC=="Brunei Darussalam"):
    atletas_original.loc[i,"Comite"]="BRU"
  elif (atleta.NOC=="Gambia"):
    atletas_original.loc[i,"Comite"]="GAM"
  elif (atleta.NOC=="Lao People's Democratic Republic"):
    atletas_original.loc[i,"Comite"]="LAO"
  elif (atleta.NOC=="St Vincent and the Grenadines"):
    atletas_original.loc[i,"Comite"]="VIN"
  elif (atleta.NOC=="Virgin Islands, US"):
    atletas_original.loc[i,"Comite"]="ISV"
  elif (atleta.NOC=="Virgin Islands, British"):
    atletas_original.loc[i,"Comite"]="IVB"
  elif (atleta.NOC=="Sao Tome and Principe"):
    atletas_original.loc[i,"Comite"]="STP"
  elif (atleta.NOC=="United Republic of Tanzania"):
    atletas_original.loc[i,"Comite"]="TAN"
  elif (atleta.NOC=="Refugee Olympic Team"):
    atletas_original.loc[i,"Comite"]="EOR"

In [ ]:
atletas_original

,Name,NOC,Discipline,Id,Comite
0,AALERUD Katrine,Norway,Cycling Road,0,NOR
1,ABAD Nestor,Spain,Artistic Gymnastics,1,ESP
2,ABAGNALE Giovanni,Italy,Rowing,2,ITA
3,ABALDE Alberto,Spain,Basketball,3,ESP
4,ABALDE Tamara,Spain,Basketball,4,ESP
...,...,...,...,...,...
11080,ZWICKER Martin Detlef,Germany,Hockey,11080,GER
11081,ZWOLINSKA Klaudia,Poland,Canoe Slalom,11081,POL
11082,ZYKOVA Yulia,ROC,Shooting,11082,ROC
11083,ZYUZINA Ekaterina,ROC,Sailing,11083,ROC


In [ ]:
paises = paises.rename({"Sigla":"Id"}, axis=1)
paises.to_csv("comites-Tokyo.csv")

## Tabela 4 - Esportes/Modalidades

In [ ]:
esporteModalidade1 = pd.DataFrame(data=atletasEsportes["Esporte"])
esporteModalidade1 = esporteModalidade1.drop_duplicates("Esporte")
esporteModalidade1 = esporteModalidade1.rename({"Esporte":"Nome"}, axis=1)
esporteModalidade1["EsportePai"] = None
esporteModalidade1.head()

,Nome,EsportePai
0,Cycling Road,None
2,Artistic Gymnastics,None
10,Rowing,None
11,Basketball,None
12,Handball,None


In [ ]:
esporteModalidade2 = pd.DataFrame(data=atletasEsportes[["Modalidade","Esporte"]])
esporteModalidade2 = esporteModalidade2.drop_duplicates()
esporteModalidade2 = esporteModalidade2.rename({"Modalidade":"Nome"}, axis=1)
esporteModalidade2 = esporteModalidade2.rename({"Esporte":"EsportePai"}, axis=1)
esporteModalidade2.head()

,Nome,EsportePai
0,Women's Individual Time Trial,Cycling Road
1,Women's Road Race,Cycling Road
2,Men's All-Around,Artistic Gymnastics
3,Men's Floor Exercise,Artistic Gymnastics
4,Men's Horizontal Bar,Artistic Gymnastics


In [ ]:
esporteModalidade = pd.concat([esporteModalidade1, esporteModalidade2])
esporteModalidade = esporteModalidade.sort_values("EsportePai")
esporteModalidade = esporteModalidade.reset_index(drop=True)
esporteModalidade["Id"] = esporteModalidade.index
esporteModalidade.head(40)

,Nome,EsportePai,Id
0,Women,3x3 Basketball,0
1,Men,3x3 Basketball,1
2,Women's Team,Archery,2
3,Mixed Team,Archery,3
4,Women's Individual,Archery,4
5,Men's Team,Archery,5
6,Men's Individual,Archery,6
7,Women's Balance Beam,Artistic Gymnastics,7
8,Women's Floor Exercise,Artistic Gymnastics,8
9,Women's Uneven Bars,Artistic Gymnastics,9


In [ ]:
esporteModalidade = esporteModalidade.fillna("-")
esporteModalidade.to_csv("esporteModalidade-Tokyo.csv")

## Tabela 7 - Comitê dos atletas

In [ ]:
comiteDosAtletas = pd.DataFrame(data=atletas_original[["Id","Comite"]])
comiteDosAtletas = comiteDosAtletas.rename({"Id":"IdAtleta"}, axis=1)
comiteDosAtletas = comiteDosAtletas.rename({"Comite":"IdComite"}, axis=1)
comiteDosAtletas.head()

,IdAtleta,IdComite
0,0,NOR
1,1,ESP
2,2,ITA
3,3,ESP
4,4,ESP


In [ ]:
comiteDosAtletas.to_csv("comiteDosAtletas-Tokyo.csv")

## Tabela 5 - Participação Comites

In [ ]:
def findNOC(x):
  nome = x.Comite
  noc = atletas_original[atletas_original['NOC']==nome].Comite.values[0]
  x.Comite = noc
  return x

medalhas = pd.read_excel("Medals.xlsx")
medalhas = medalhas.drop(columns=["Total","Rank by Total"])
medalhas = medalhas.rename({"Team/NOC":"Comite"}, axis=1)
medalhas = medalhas.rename({"Rank":"Classificacao"}, axis=1)
medalhas = medalhas.rename({"Gold":"QtdOuro"}, axis=1)
medalhas = medalhas.rename({"Silver":"QtdPrata"}, axis=1)
medalhas = medalhas.rename({"Bronze":"QtdBronze"}, axis=1)

medalhas = medalhas.apply(lambda x: findNOC(x), axis=1)

In [ ]:
medalhas1 = pd.read_excel("Medals.xlsx")
medalhas1.head()

,Rank,Team/NOC,Gold,Silver,Bronze,Total,Rank by Total
0,1,United States of America,39,41,33,113,1
1,2,People's Republic of China,38,32,18,88,2
2,3,Japan,27,14,17,58,5
3,4,Great Britain,22,21,22,65,4
4,5,ROC,20,28,23,71,3


In [ ]:
paisesSemMedalha = pd.DataFrame(columns=["Comite"])
paisesAux = pd.DataFrame(data=atletas_original["Comite"])
paisesAux = paisesAux.drop_duplicates("Comite")
paisesAux = paisesAux.reset_index(drop=True)

for i in range(len(paisesAux)):
  pais = paisesAux.loc[i]
  if not (pais.Comite in medalhas["Comite"].unique()):
    paisesSemMedalha.loc[len(paisesSemMedalha)] = [pais.Comite]

In [ ]:
paisesSemMedalha["QtdOuro"] = 0
paisesSemMedalha["QtdPrata"] = 0
paisesSemMedalha["QtdBronze"] = 0
paisesSemMedalha["Classificacao"] = "-"
paisesSemMedalha

,Comite,QtdOuro,QtdPrata,QtdBronze,Classificacao
0,CHI,0,0,0,-
1,SUD,0,0,0,-
2,SGP,0,0,0,-
3,MDV,0,0,0,-
4,MLT,0,0,0,-
...,...,...,...,...,...
107,TUV,0,0,0,-
108,IVB,0,0,0,-
109,CAF,0,0,0,-
110,MYA,0,0,0,-


In [ ]:
medalhasAux = pd.concat([medalhas,paisesSemMedalha])
medalhasAux = medalhasAux.reset_index(drop=True)
medalhasAux

,Classificacao,Comite,QtdOuro,QtdPrata,QtdBronze
0,1,USA,39,41,33
1,2,CHN,38,32,18
2,3,JPN,27,14,17
3,4,GBR,22,21,22
4,5,ROC,20,28,23
...,...,...,...,...,...
200,-,TUV,0,0,0
201,-,IVB,0,0,0
202,-,CAF,0,0,0
203,-,MYA,0,0,0


In [ ]:
medalhas = medalhasAux.rename({"Comite":"IdComite"}, axis=1)
medalhas["AnoEdicao"] = 2020    #Aqui é só sobre 2020 mesmo
medalhas["QtdAtletas"] = 0
participacaoComites = medalhas.reindex(columns=["IdComite","AnoEdicao","QtdAtletas","QtdOuro","QtdPrata","QtdBronze","Classificacao"])

In [ ]:
participacaoComites

,IdComite,AnoEdicao,QtdAtletas,QtdOuro,QtdPrata,QtdBronze,Classificacao
0,USA,2020,0,39,41,33,1
1,CHN,2020,0,38,32,18,2
2,JPN,2020,0,27,14,17,3
3,GBR,2020,0,22,21,22,4
4,ROC,2020,0,20,28,23,5
...,...,...,...,...,...,...,...
200,TUV,2020,0,0,0,0,-
201,IVB,2020,0,0,0,0,-
202,CAF,2020,0,0,0,0,-
203,MYA,2020,0,0,0,0,-


In [ ]:
def findQtdAtletas(row):
  pais = row.IdComite
  qtd = len(comiteDosAtletas[comiteDosAtletas['IdComite']==pais])
  row.QtdAtletas = qtd
  return row

participacaoComites = participacaoComites.apply(lambda x: findQtdAtletas(x), axis=1)
participacaoComites

,IdComite,AnoEdicao,QtdAtletas,QtdOuro,QtdPrata,QtdBronze,Classificacao
0,USA,2020,615,39,41,33,1
1,CHN,2020,401,38,32,18,2
2,JPN,2020,586,27,14,17,3
3,GBR,2020,366,22,21,22,4
4,ROC,2020,318,20,28,23,5
...,...,...,...,...,...,...,...
200,TUV,2020,2,0,0,0,-
201,IVB,2020,3,0,0,0,-
202,CAF,2020,2,0,0,0,-
203,MYA,2020,2,0,0,0,-


In [ ]:
participacaoComites.to_csv("participacaoComites-Tokyo.csv")

## Tabela 6 - Participação Atletas

In [ ]:
participacaoAtletas = pd.DataFrame(columns=["IdAtleta","AnoEdicao","IdModalidade","Altura","Peso","Medalha"])

In [ ]:
for i in range(len(atletasEsportes)):
  atleta = atletasEsportes.loc[i]
  nome = atleta.Nome
  esporte = atleta.Esporte
  modalidade = atleta.Modalidade
  idAtleta = atletas[atletas['Nome']==nome].Id.values[0]
  idModalidade = esporteModalidade.loc[((esporteModalidade['Nome']==modalidade) & (esporteModalidade['EsportePai']==esporte))].Id.values[0]
  if atleta.Medalha=='G':
    medalha="Gold"
  elif atleta.Medalha=='S':
    medalha="Silver"
  elif atleta.Medalha=='B':
    medalha="Bronze"
  else:
    medalha="-"
  participacaoAtletas.loc[len(participacaoAtletas)] = [idAtleta, 2020, idModalidade, "-","-",medalha]
  

In [ ]:
participacaoAtletas.head(30)

,IdAtleta,AnoEdicao,IdModalidade,Altura,Peso,Medalha
0,0,2020,120,-,-,-
1,0,2020,118,-,-,-
2,1,2020,18,-,-,-
3,1,2020,19,-,-,-
4,1,2020,20,-,-,-
5,1,2020,13,-,-,-
6,1,2020,17,-,-,-
7,1,2020,16,-,-,-
8,1,2020,15,-,-,-
9,1,2020,14,-,-,-


In [ ]:
participacaoAtletas.to_csv("participacaoAtletas-Tokyo.csv")

## Tabela 8 - Esportes das edições

In [ ]:
esportesDasEdicoes=pd.DataFrame(data=esporteModalidade['Id'])
esportesDasEdicoes["AnoEdicao"]=2020
esportesDasEdicoes["Ouro"] = None
esportesDasEdicoes["Prata"]=None
esportesDasEdicoes["Bronze"]=None

def setPodio(row):  
  try:
    infoPodioOuro=participacaoAtletas.loc[(participacaoAtletas['IdModalidade'] == row.Id) & (participacaoAtletas['Medalha']=="Gold")]
    infoPodioOuro=infoPodioOuro.reset_index(drop=True)
    index = int(infoPodioOuro.IdAtleta.values[0])
    row.Ouro=comiteDosAtletas[comiteDosAtletas['IdAtleta']==index].IdComite.values[0]
  except:
    row.Ouro="-"

  try: 
    infoPodioPrata=participacaoAtletas.loc[(participacaoAtletas['IdModalidade'] == row.Id) & (participacaoAtletas['Medalha']=="Silver")]
    infoPodioPrata=infoPodioPrata.reset_index(drop=True)
    index = int(infoPodioPrata.IdAtleta.values[0])
    row.Prata=comiteDosAtletas[comiteDosAtletas['IdAtleta']==index].IdComite.values[0]
  except:
    row.Prata="-"

  try:
    infoPodioBronze=participacaoAtletas.loc[(participacaoAtletas['IdModalidade'] == row.Id) & (participacaoAtletas['Medalha']=="Bronze")]
    infoPodioBronze=infoPodioBronze.reset_index(drop=True)
    index = int(infoPodioBronze.IdAtleta.values[0])
    row.Bronze=comiteDosAtletas[comiteDosAtletas['IdAtleta']==index].IdComite.values[0]
  except:
    row.Bronze="-"

  return row


esportesDasEdicoes = esportesDasEdicoes.apply(lambda x: setPodio(x),axis=1)  #Aplico a função acima para cada linha. ('lambda x' é tipo 'para cada linha x')

esportesDasEdicoes=esportesDasEdicoes.rename({"Id":"IdModalidade"}, axis=1)

esportesDasEdicoes=esportesDasEdicoes.sort_values(by=['AnoEdicao','IdModalidade'])
esportesDasEdicoes = esportesDasEdicoes.reset_index(drop=True)

In [ ]:
esportesDasEdicoes = esportesDasEdicoes[esportesDasEdicoes.index<339]

In [ ]:
esportesDasEdicoes.head(50)

,IdModalidade,AnoEdicao,Ouro,Prata,Bronze
0,0,2020,USA,ROC,CHN
1,1,2020,LAT,ROC,SRB
2,2,2020,KOR,ROC,GER
3,3,2020,KOR,NED,MEX
4,4,2020,KOR,ROC,ITA
5,5,2020,KOR,TPE,JPN
6,6,2020,TUR,-,JPN
7,7,2020,CHN,-,USA
8,8,2020,USA,ITA,ROC
9,9,2020,BEL,ROC,USA


In [ ]:
esportesDasEdicoes.to_csv("esportesDasEdicoes-Tokyo.csv")